In [ ]:
import numpy as np

from ptable_dict import ptable, atomic_masses
from utilities import write_xyz, load_xyz

# calculate vectors for large MD box

In [ ]:
# 147.2122 x 137.2347 x 102.7109 (abc)
# 85.00009, 78.38998, 59.10033 (alpha, beta, gamma)
def calc_real_space_abc(a_mag, b_mag, c_mag, alpha_deg, beta_deg, gamma_deg):
    '''
    https://www.ucl.ac.uk/~rmhajc0/frorth.pdf
    '''
    alpha = np.deg2rad(alpha_deg)
    beta = np.deg2rad(beta_deg)
    gamma = np.deg2rad(gamma_deg)
    
    V = a_mag*b_mag*c_mag*np.sqrt(1-np.cos(alpha)**2-np.cos(beta)**2-np.cos(gamma)**2+2*np.cos(alpha)*np.cos(beta)*np.cos(gamma)) 
    
    ax = a_mag
    ay = 0
    az = 0
    a = np.array([ax, ay, az])
    
    bx = b_mag*np.cos(gamma)
    by = b_mag*np.sin(gamma)
    bz = 0
    b = np.array([bx, by, bz])
    
    cx = c_mag*np.cos(beta)
    cy = c_mag*(np.cos(alpha)-np.cos(beta)*np.cos(gamma))/(np.sin(gamma))
    cz = V/(a_mag*b_mag*np.sin(gamma))
    c = np.array([cx, cy, cz])
    
    return a, b, c

# Use vectors calculated above to repeat MD box

In [ ]:
import os

from ptable_dict import ptable, atomic_masses
from utilities import write_xyz, load_xyz, calc_real_space_abc

# Inputs
name = 'F3_new2'
x_size = 300
y_size = 80
z_size = 300
a,b,c = (139.440, 67.318, 73.529)
alpha,beta,gamma = (83.31, 71.10, 74.75)

# dirr = os.getcwd()
dirr = '/Users/Thomas2/Library/CloudStorage/OneDrive-UCB-O365/Desktop/Research_Stuff/OPV_GIWAXS/N2200_simulation/xyz_files/'
import numpy as np
xyz_file = f'{dirr}{name}.xyz'
coords, elements = load_xyz(xyz_file)

a_vect, b_vect, c_vect = calc_real_space_abc(a, b, c, alpha, beta, gamma)
num_x = int(np.ceil(2*x_size/a_vect[0]))
num_y = int(np.ceil(2*y_size/b_vect[1]))
num_z = int(np.ceil(2*z_size/c_vect[2]))

print(f'num_x:{num_x}\nnum_y:{num_y}\nnum_z:{num_z}')

coords_original = coords
elements_original = elements
for i in range(num_x):
    coords_new = coords_original+[a_vect[0]*(i+1), 0, 0]
    if i == 0:
        coords_append = coords_new
        elements_append = elements_original
    else:
        coords_append = np.concatenate((coords_append, coords_new), axis=0)
        elements_append = np.concatenate((elements_append, elements), axis=0)  
    if i==num_x-1:
        coords = np.concatenate((coords_original, coords_append), axis=0)
        elements = np.concatenate((elements_original, elements_append), axis=0)

coords_original = coords
elements_original = elements
for i in range(num_y):
    coords_new = coords_original+[b_vect[0]*(i+1), b_vect[1]*(i+1), 0]
    if i ==0:
        coords_append = coords_new
        elements_append = elements_original
    else:
        coords_append = np.concatenate((coords_append, coords_new), axis=0)
        elements_append = np.concatenate((elements_append, elements), axis=0)
    if i==num_y-1:
        coords = np.concatenate((coords_original, coords_append), axis=0)
        elements = np.concatenate((elements_original, elements_append), axis=0)

coords_original = coords
elements_original = elements
for i in range(num_z):
    coords_new = coords_original + [c_vect[0]*(i+1),  c_vect[1]*(i+1), c_vect[2]*(i+1)]
    if i ==0:
        coords_append = coords_new
        elements_append = elements_original
    else:
        coords_append = np.concatenate((coords_append, coords_new), axis=0)
        elements_append = np.concatenate((elements_append, elements), axis=0)
    if i==num_z-1:
        coords = np.concatenate((coords_original, coords_append), axis=0)
        elements = np.concatenate((elements_original, elements_append), axis=0)

del coords_original
del elements_original
del coords_append
del elements_append

x_max = np.max(coords[:,0])-np.min(coords[:,0])
y_max = np.max(coords[:,1])-np.min(coords[:,1])
z_max = np.max(coords[:,2])-np.min(coords[:,2])

x_buffer = (x_max-x_size)/2
y_buffer = (y_max-y_size)/2
z_buffer = (z_max-z_size)/2

x_lower = x_buffer
x_upper = x_max-x_buffer
y_lower = y_buffer
y_upper = y_max-y_buffer
z_lower = z_buffer
z_upper = z_max-z_buffer

# Shift coords array to origin 
coords[:,0] -= np.min(coords[:,0])
coords[:,1] -= np.min(coords[:,1])
coords[:,2] -= np.min(coords[:,2])

# Use NumPy masking to filter the coordinates
mask = (
    (coords[:,0] >= x_lower) & (coords[:,0] <= x_upper) &
    (coords[:,1] >= y_lower) & (coords[:,1] <= y_upper) &
    (coords[:,2] >= z_lower) & (coords[:,2] <= z_upper)
)

coords_new = coords[mask]
elements_new = elements[mask]

save_path = f'{dirr}{name}_slab_{x_size}x{y_size}x{z_size}.xyz'
write_xyz(save_path, coords_new, elements_new)

In [ ]:
dirr = '/Users/Thomas2/Library/CloudStorage/OneDrive-UCB-O365/Desktop/Research_Stuff/OPV_GIWAXS/N2200_simulation/Form2/'
# names = ['dump_10', 'dump_20', 'dump_30', 'dump_40', 'dump_50', 'dump_60', 'dump_70', 'dump_80', 'dump_90', 'dump_100']
names = ['Form2']

# number of adds in each direction
# num_x = 4
# num_y = 3
# num_z = 4

num_x = 2
num_y = 2
num_z = 2

#Unit cell params
#misc params
# a,b,c = (14.1990, 6.8549, 7.4873)
# alpha,beta,gamma = (83.31, 71.10, 74.75)

# a,b,c = (28.032282, 4.51059, 24.635977)
# alpha,beta,gamma = (83.314751, 71.098457, 74.751366)

#Form1 alpha
# a,b,c = (15.07653, 6.086625558, 7.759215338) #nm
# alpha, beta, gamma = (75.80018168, 69.91988911, 86.0504032) #nm

# Form2
a,b,c = (14.5780, 11.7355, 10.2309) #nm
alpha, beta, gamma = (89.13, 88.00, 87.50) #nm
# a,b,c = (14.578, 7.824, 25.577)
# alpha, beta, gamma = (89.13, 88, 87.5)

# Form3
# a,b,c = ( 13.9412, 13.5789, 10.0333) #nm
# alpha, beta, gamma = (83.98, 71.99, 73.18) #nm

# #Form4
# a,b,c = (14.7212, 13.7235, 10.2711) #nm
# alpha, beta, gamma = (85.00009, 78.38998, 59.10033) #nm

a_vect, b_vect, c_vect = calc_real_space_abc(a, b, c, alpha, beta, gamma)
a_vect *= 10
b_vect *= 10
c_vect *= 10

for name in names:
    xyz_file = f'{dirr}{name}.xyz'
    coords, elements = load_xyz(xyz_file)
    
    x_max = np.max(coords[:,0])-np.min(coords[:,0])
    y_max = np.max(coords[:,1])-np.min(coords[:,1])
    z_max = np.max(coords[:,2])-np.min(coords[:,2])
    
    coords_original = coords
    elements_original = elements
    for i in range(num_x):
        coords_new = coords_original+[a_vect[0]*(i+1), 0, 0]
        if i == 0:
            coords_append = coords_new
            elements_append = elements_original
        else:
            coords_append = np.concatenate((coords_append, coords_new), axis=0)
            elements_append = np.concatenate((elements_append, elements), axis=0)  
        if i==num_x-1:
            coords = np.concatenate((coords_original, coords_append), axis=0)
            elements = np.concatenate((elements_original, elements_append), axis=0)
    
    coords_original = coords
    elements_original = elements
    for i in range(num_y):
        coords_new = coords_original+[b_vect[0]*(i+1), b_vect[1]*(i+1), 0]
        if i ==0:
            coords_append = coords_new
            elements_append = elements_original
        else:
            coords_append = np.concatenate((coords_append, coords_new), axis=0)
            elements_append = np.concatenate((elements_append, elements), axis=0)
        if i==num_y-1:
            coords = np.concatenate((coords_original, coords_append), axis=0)
            elements = np.concatenate((elements_original, elements_append), axis=0)
    
    coords_original = coords
    elements_original = elements
    for i in range(num_z):
        coords_new = coords_original + [c_vect[0]*(i+1),  c_vect[1]*(i+1), c_vect[2]*(i+1)]
        if i ==0:
            coords_append = coords_new
            elements_append = elements_original
        else:
            coords_append = np.concatenate((coords_append, coords_new), axis=0)
            elements_append = np.concatenate((elements_append, elements), axis=0)
        if i==num_z-1:
            coords = np.concatenate((coords_original, coords_append), axis=0)
            elements = np.concatenate((elements_original, elements_append), axis=0)
    
    save_path = f'{dirr}{name}_{num_x+1}x{num_y+1}x{num_z+1}.xyz'
    write_xyz(save_path, coords, elements)

    del coords, coords_append, coords_new, coords_original
    del elements, elements_append, elements_original

# slice box into rectangular prism

In [ ]:
#make rectangular prism
dirr = '/Users/Thomas2/Library/CloudStorage/OneDrive-UCB-O365/Desktop/Research_Stuff/OPV_GIWAXS/N2200_simulation/Form2/'
# # names = ['dump_10', 'dump_20', 'dump_30', 'dump_40', 'dump_50', 'dump_60', 'dump_70', 'dump_80', 'dump_90', 'dump_100']
names = ['Form2']
for name in names:
    # xyz_file = f'{dirr}{name}_{num_x+1}x{num_y+1}x{num_z+1}.xyz'
    xyz_file = f'{dirr}{name}.xyz'
    coords, elements = load_xyz(xyz_file)
    
    x_max = np.max(coords[:,0])-np.min(coords[:,0])
    y_max = np.max(coords[:,1])-np.min(coords[:,1])
    z_max = np.max(coords[:,2])-np.min(coords[:,2])
    
    x_size = 150
    y_size = 80
    z_size = 30
    # x_size = 30
    # y_size = 10
    # z_size = 30

    assert x_max>x_size
    assert y_max>y_size
    assert z_max>z_size

    x_buffer = (x_max-x_size)/2
    y_buffer = (y_max-y_size)/2
    z_buffer = (z_max-z_size)/2


    x_lower = x_buffer
    x_upper = x_max-x_buffer
    y_lower = y_buffer
    y_upper = y_max-y_buffer
    z_lower = z_buffer
    z_upper = z_max-z_buffer
    
    # Shift coords array to origin 
    coords[:,0] -= np.min(coords[:,0])
    coords[:,1] -= np.min(coords[:,1])
    coords[:,2] -= np.min(coords[:,2])
    
    # Use NumPy masking to filter the coordinates
    mask = (
        (coords[:,0] >= x_lower) & (coords[:,0] <= x_upper) &
        (coords[:,1] >= y_lower) & (coords[:,1] <= y_upper) &
        (coords[:,2] >= z_lower) & (coords[:,2] <= z_upper)
    )
    
    coords_new = coords[mask]
    elements_new = elements[mask]
    
    
    save_path = f'{dirr}{name}_rect_cut{x_size}x{y_size}x{z_size}.xyz'
    write_xyz(save_path, coords_new, elements_new)

# Sum multiple .xyz files

In [ ]:
dirr = '/Users/Thomas2/Library/CloudStorage/OneDrive-UCB-O365/Desktop/Research_Stuff/OPV_GIWAXS/N2200_simulation/dump_files/'
names = ['dump_10', 'dump_20', 'dump_30', 'dump_40', 'dump_50', 'dump_60', 'dump_70', 'dump_80', 'dump_90', 'dump_100']

for i, name in enumerate(names):
    xyz_file = f'{dirr}{name}_rect_cut125x25x10.xyz'
    coords, elements = load_xyz(xyz_file)
    if i==0:
        coords_append = coords
        elements_append = elements
    else:
        coords_append = np.concatenate((coords_append, coords), axis=0)
        elements_append = np.concatenate((elements_append, elements), axis=0)

save_path = f'{dirr}sum_rect_cut125x25x10.xyz'
write_xyz(save_path, coords_append, elements_append)